In [1]:
# !pip install geopy

In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
import csv
from geopy.extra.rate_limiter import RateLimiter
import concurrent.futures

geolocator = Nominatim(user_agent="your_app_name", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [3]:

from geopy.exc import GeocoderQueryError

def get_coord(address):
    try:
        location = geolocator.geocode(address)
        if location:
            print(location.address)
            return location.latitude, location.longitude
        else:
            # Handle the case where the location is not found
            return None, None
    except GeocoderQueryError as e:
        # Handle the geocoding error
        print(f"Geocoding error: {e}")
        return None, None

In [4]:
import numpy as np
def get_detail_add():
    data_file_path = "/home/nmh/ds/project/data_validation/cleaned_concatenated_data.csv"
    data = pd.read_csv(data_file_path)
    detailed_data = []

    for index, row in data.iterrows():
        address_components = [row['street'], row['ward'], row['district'], row['city']]
        address = ' '.join(filter(None, address_components))  # Join non-empty components
        detailed_data.append((row, address))

    return detailed_data

In [5]:
# !rm -r /kaggle/working/output.csv

In [6]:
import time
def process_row(row_data):
    try:
        row, address = row_data
        lat, lon = get_coord(address)
        return row.tolist() + [lat, lon]
    except Exception as e:
        print(f"Exception occurred while processing address {address}: {e}")
        return address, None, None

In [7]:
def main():
    detailed_data = get_detail_add()
    csv_file_path = '/home/nmh/ds/project/data_coordinating/data_with_coordination_ver2.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header
        header = ['','upload_date','house_direction', 'bedroom', 'toilet', 'legits', 'furniture', 
                  'floors', 'facade', 'entrance', 'city', 'district', 'ward', 
                  'street', 'area', 'price', 'latitude', 'longitude']
        writer.writerow(header)

        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            future_to_row = {executor.submit(process_row, row_data): row_data for row_data in detailed_data}

            for future in concurrent.futures.as_completed(future_to_row):
                try:
                    result = future.result()
                    writer.writerow(result)
                except Exception as e:
                    print(f"Exception occurred while processing a row: {e}")

In [8]:
if __name__ == "__main__":
    main()

Đường Quang Trung, Phường Yết Kiêu, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Chung cư Hà Nội Homeland Long Biên, Đường nội bộ, Phường Thượng Thanh, Quận Long Biên, Thành phố Hà Nội, 08443, Việt Nam
Đường Trần Phú, Phường Văn Quán, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Đường Quang Trung, Phường Yết Kiêu, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Đường Lê Quang Đạo, Phường Phú Đô, Quận Nam Từ Liêm, Thành phố Hà Nội, 12011, Việt Nam
Phố Quan Nhân, Phường Nhân Chính, Quận Thanh Xuân, Thành phố Hà Nội, 11513, Việt Nam
Đường Trung Văn, Phường Trung Văn, Quận Nam Từ Liêm, Thành phố Hà Nội, 12011, Việt Nam
Đường Nghi Tàm, Phường Yên Phụ, Quận Tây Hồ, Thành phố Hà Nội, 11207, Việt Nam
Đường Nguyễn Văn Huyên, Phường Nghĩa Đô, Quận Cầu Giấy, Thành phố Hà Nội, 11150, Việt Nam
Phố Lê Thanh Nghị, Phường Đồng Tâm, Quận Hai Bà Trưng, Thành phố Hà Nội, 10999, Việt Nam
Đường Hoàng Hoa Thám, Phường Ngọc Hà, Quận Ba Đình, Thành phố Hà Nội, 11112, Việt Nam
Đường Nguyễn Khánh Toàn, Ph